# Imports

In [3]:
import xgboost as xgb
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Train

In [4]:
train_set = pd.read_csv('train/train_encoded.csv')
train_set.head(10)

,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,keyword_median,...,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v,target
0,0,0,43,69,26,1.0,1.0,1.0,1.0,1.0,...,0.224894,0.474996,0.343243,0.323476,8.481658,0.000850,0.615265,0.154212,0.121727,1
1,0,0,36,38,2,1.0,1.0,1.0,1.0,1.0,...,0.211650,0.508967,0.363618,0.340166,0.790154,-0.103024,0.308946,0.079015,0.055061,1
2,0,0,72,133,61,1.0,1.0,1.0,1.0,1.0,...,0.153758,0.585124,0.327084,0.256037,29.265612,-0.044882,1.000000,0.139360,0.095317,1
3,0,0,50,65,15,1.0,1.0,1.0,1.0,1.0,...,0.150085,0.574226,0.359336,0.346962,2.733583,-0.039545,0.317247,0.097628,0.082558,1
4,0,0,52,88,36,1.0,1.0,1.0,1.0,1.0,...,0.247769,0.444024,0.326443,0.287533,13.153737,-0.041156,1.000000,0.125274,0.086994,1
5,0,0,84,110,26,1.0,1.0,1.0,1.0,1.0,...,0.094812,0.392155,0.267090,0.268381,7.810325,-0.093557,0.478663,0.100132,0.080587,1
6,0,0,74,95,21,1.0,1.0,1.0,1.0,1.0,...,0.211903,0.421501,0.304355,0.292368,9.341228,-0.047981,0.788929,0.141534,0.109943,1
7,0,0,21,59,38,1.0,1.0,1.0,1.0,1.0,...,0.288043,0.580492,0.433880,0.410864,11.859967,-0.143466,1.000000,0.152051,0.112546,1
8,0,0,38,79,41,1.0,1.0,1.0,1.0,1.0,...,0.344773,0.491582,0.404259,0.389401,15.609978,-0.033696,1.000000,0.200128,0.158737,1
9,0,0,24,52,28,1.0,1.0,1.0,1.0,1.0,...,0.391599,0.639305,0.491594,0.467736,7.238668,0.004347,0.526257,0.160859,0.136866,1


### XGBoost

In [5]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Random Search

In [6]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [7]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [8]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [9]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 20,
 'learning_rate': 0.01,
 'colsample_bytree': 0.4}

In [10]:
preds = xgb_random.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.788652


In [11]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4,\
                           learning_rate = 0.001, max_depth = 50, n_estimators = 50)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.784520


In [12]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.794395


In [13]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.792905


##### Best params de random forest

In [14]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.7,\
                           learning_rate = 0.1, max_depth = 37, n_estimators = 60)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.781906


##### Parámetros que mejor funcionaron con pruebas a mano

In [15]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.781333


# Test

In [16]:
test_set = pd.read_csv('test/test_encoded.csv')

In [17]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [29]:
preds = xg_reg.predict(test_set)
preds

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
test_set['target'] = preds
test_set['id'] = col_id
test_set[['id', 'target']]

### Resultado

In [30]:
test_set[['id', 'target']].to_csv('result.csv', header=True, index=False)